In [1]:
from pymatgen import Structure, MPRester, Lattice
# from math import sin, cos, sqrt, pi
from pymatgen.core.surface import SlabGenerator, Slab
from pymatgen.vis.structure_vtk import StructureVis
from atomate.vasp.fireworks.core import OptimizeFW
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer as SGA
mpr = MPRester('xGTZsWPN3BoydaL4')
from pymatgen.io.vasp.inputs import Poscar

/Users/jordanburns/miniconda3/envs/dev/lib/python3.7/site-packages/pymatgen/io/cif.py:44: UserWarning: Please install optional dependency pybtex if youwant to extract references from CIF files.
  warnings.warn("Please install optional dependency pybtex if you"


In [37]:
# import Li2MnO3 structure from materials project 
# struct = mpr.get_structure_by_material_id("mp-18988", conventional_unit_cell=True)

# try doing wyckoff numbers from relaxed structure instead
from maggma.stores import MongoStore 
from pandas import DataFrame
tasks = MongoStore("fw_jb_vasp", "tasks", host="mongodb03.nersc.gov", username="admin_jb_vasp", password="u7dvKmA^")
tasks.connect()




ServerSelectionTimeoutError: mongodb03.nersc.gov:27017: [Errno 61] Connection refused

In [25]:
struct = Structure.from_file(filename='Li2MnO3_2.CONTCAR.txt')

In [29]:
sga = SGA(struct,0.01)

In [30]:
struct_sym = sga.get_symmetrized_structure()

In [31]:
wyckoff_symbols = struct_sym.wyckoff_symbols
print(wyckoff_symbols)

['8f', '4e', '4c', '4e', '4e', '8f', '8f', '8f']


In [32]:
equiv_indices = struct_sym.equivalent_indices
wyckoff_labels = ['']*len(struct_sym)
for wyckoff_symbol, equiv_group in zip(wyckoff_symbols, equiv_indices):
    for idx in equiv_group:
        wyckoff_labels[idx] = wyckoff_symbol
print(wyckoff_labels)

['8f', '4e', '8f', '8f', '8f', '4e', '4c', '4c', '8f', '4e', '8f', '8f', '8f', '4e', '4c', '4c', '4e', '4e', '4e', '4e', '4e', '4e', '4e', '4e', '8f', '8f', '8f', '8f', '8f', '8f', '8f', '8f', '8f', '8f', '8f', '8f', '8f', '8f', '8f', '8f', '8f', '8f', '8f', '8f', '8f', '8f', '8f', '8f']


In [33]:
struct_sym.add_site_property('wyckoff', wyckoff_labels)
struct_sym = Structure.from_sites(struct_sym.sites)

In [34]:
slabgen_wyckoff = SlabGenerator(struct_sym, [0,0,1], 10 , 10, in_unit_planes=True)
slabs_wyckoff = slabgen_wyckoff.get_slabs()
all_slabs_corrected_wyckoff = [slabgen_wyckoff.nonstoichiometric_symmetrized_slab(slab)
                               for slab in slabs_wyckoff]
all_slabs_corrected_wyckoff = sum(all_slabs_corrected_wyckoff, [])

In [35]:
##Now get charge attributes for ions

In [36]:
bva_struct = bva_analyzer.get_oxi_state_decorated_structure(struct_sym)

NameError: name 'bva_analyzer' is not defined

In [ ]:
slabgen_bva_before_sym = SlabGenerator(bva_struct, [0,0,1], 10 , 10, in_unit_planes=True)
slabs_bva_before = slabgen_bva_before_sym.get_slabs()

In [ ]:
slabgen_bva = SlabGenerator(bva_struct, [1,1,0], 10 , 10, in_unit_planes=True)
slabs_bva = slabgen_bva.get_slabs()
all_slabs_corrected_bva = [slabgen_wyckoff.nonstoichiometric_symmetrized_slab(slab)
                               for slab in slabs_bva]
all_slabs_corrected_bva = sum(all_slabs_corrected_bva, [])